In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import pytfa
import numpy as np
from optlang.exceptions import SolverError

#from cobra.core.model import SolverNotFound
from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data
from pytfa.optim.relaxation import relax_dgo

import pickle
from pytfa.optim.variables import LogConcentration
#thermo_database = '/projectnb2/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/data/thermo_data.thermodb'
#root_dir = '/projectnb/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/tests/singleCoenzymeModel.08272021.v2'
thermo_database = '../assets/thermo_data.thermodb'
model_path = '../assets'

CPLEX = 'optlang-cplex'
GUROBI = 'optlang-gurobi'
GLPK = 'optlang-glpk'
solver = GUROBI

# Load reaction DB
print("Loading thermo data...")
thermo_data = load_thermoDB(thermo_database)

print("Done !")
#biomass_rxn = 'BIOMASS_Ec_iJO1366_WT_53p95M'
biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'


model_id = 'iJO1366_NAD_semi-unconstrained.json'
#model_id = 'iJO1366.json'

# We import pre-compiled data as it is faster for bigger models
cobra_model = load_json_model(model_path + '/' + model_id)

lexicon = read_lexicon(model_path + '/iJO1366/lexicon.csv')
compartment_data = read_compartment_data(model_path + '/iJO1366/compartment_data.json')

# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

# Annotate the cobra_model
annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)


mytfa.name = 'iJO1366[NAD]'
mytfa.solver = solver
mytfa.objective = biomass_rxn

# Solver settings

def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)


## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
print('FBA Solution found : {0:.5g}'.format(fba_value))

# fva = flux_variability_analysis(mytfa)

## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

## Info on the cobra_model
mytfa.print_info()

## Optimality
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value
print('TFA Solution found : {0:.5g}'.format(tfa_value))

# It might happen that the model is infeasible. In this case, we can relax 
# thermodynamics constraints:

if tfa_value < 0.1:
    
    from pytfa.optim.relaxation import relax_dgo
    #mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)
    original_model, mytfa = mytfa, relaxed_model
    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

    print('relaxed TFA Solution found : {0:.5g}'.format(tfa_value))

# Report
print('FBA Solution found : {0:.5g}'.format(fba_value))
print('TFA Solution found : {0:.5g}'.format(tfa_value))


single_coenzyme_models = {}
z = mytfa.copy()
single_coenzyme_models['tmodel'] = z
single_coenzyme_models['tmodel-maxGrowth'] = tfa_value
single_coenzyme_models['tmodel-solution'] = tfa_solution

Academic license - for non-commercial use only - expires 2022-08-19
Using license file /Users/joshuagoldford/gurobi.lic
Loading thermo data...
Done !
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp8z_pv9qa.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20294 nonzeros


2021-10-11 09:46:49,093 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-11 09:46:49,320 - thermomodel_None - INFO - # Model preparation starting...


FBA Solution found : 24.714


2021-10-11 09:46:52,200 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 09:46:52,201 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,202 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,203 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 09:46:52,204 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,205 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,356 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 09:46:52,357 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,357 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,358 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 09:46:52,359 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,361 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 09:46:52,425 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 09:46:52,426 - thermomodel_None - WARNING - Warning :

                        value
key                          
name             iJO1366[NAD]
description      iJO1366[NAD]
num constraints         15014
num variables           15518
num metabolites          1805
num reactions            2582
                             value
key                               
num metabolites(thermo)       1548
num reactions(thermo)         1821
pct metabolites(thermo)  85.761773
pct reactions(thermo)    70.526723
TFA Solution found : 0
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp5o6uz7yl.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20264 nonzeros


2021-10-11 09:47:57,392 - thermomodel_iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpvhcrtzgg.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20264 nonzeros


2021-10-11 09:48:13,604 - thermomodel_iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-11 09:48:22,831 - thermomodel_iJO1366[NAD] - INFO - Adding slack constraints
adding slacks: 100%|██████████| 1821/1821 [00:50<00:00, 36.28it/s]
2021-10-11 09:49:14,769 - thermomodel_iJO1366[NAD] - INFO - Optimizing slack model
2021-10-11 09:49:23,845 - thermomodel_iJO1366[NAD] - INFO - Extracting relaxation
applying slack: 100%|██████████| 2582/2582 [00:00<00:00, 100784.46it/s]
2021-10-11 09:49:23,977 - thermomodel_iJO1366[NAD] - INFO - Testing relaxation


Relaxation: 
              lb_in      ub_in  lb_change  ub_change     lb_out     ub_out
AIRC3    -11.941085  -8.988685   0.000000   2.582409 -11.941085  -6.406276
ARGSL     16.770653  18.636729   2.184361   0.000000  14.586293  18.636729
ATPS4rpp  -9.518201  -8.083092   1.517942   0.000000 -11.036143  -8.083092
GALT1     67.754735  69.018800  54.942183   0.000000  12.812552  69.018800
HEPK2     44.872230  45.345690  32.059679   0.000000  12.812552  45.345690
HEPT1     25.810459  27.074524  13.693847   0.000000  12.116612  27.074524
MECDPS    26.135683  30.646876  11.549390   0.000000  14.586293  30.646876
MTHTHFSs  13.297862  14.814659   6.891586   0.000000   6.406276  14.814659
relaxed TFA Solution found : 0.72479
FBA Solution found : 24.714
TFA Solution found : 0.72479
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpa1n8j0gr.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20264 nonzeros


2021-10-11 09:49:35,078 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [47]:
help(relax_dgo)

Help on function relax_dgo in module pytfa.optim.relaxation:

relax_dgo(tmodel, reactions_to_ignore=(), solver=None, in_place=False)
    :param t_tmodel:
    :type t_tmodel: pytfa.thermo.ThermoModel:
    :param reactions_to_ignore: Iterable of reactions that should not be relaxed
    :param solver: solver to use (e.g. 'optlang-glpk', 'optlang-cplex',
        'optlang-gurobi'
    :return: a cobra_model with relaxed bounds on standard Gibbs free energy



In [48]:
help(pytfa.optim.relaxation)

Help on module pytfa.optim.relaxation in pytfa.optim:

NAME
    pytfa.optim.relaxation

DESCRIPTION
    .. module:: pytfa
       :platform: Unix, Windows
       :synopsis: Thermodynamics-based Flux Analysis
    
    .. moduleauthor:: pyTFA team
    
    Relaxation of models with constraint too tight

FUNCTIONS
    relax_dgo(tmodel, reactions_to_ignore=(), solver=None, in_place=False)
        :param t_tmodel:
        :type t_tmodel: pytfa.thermo.ThermoModel:
        :param reactions_to_ignore: Iterable of reactions that should not be relaxed
        :param solver: solver to use (e.g. 'optlang-glpk', 'optlang-cplex',
            'optlang-gurobi'
        :return: a cobra_model with relaxed bounds on standard Gibbs free energy
    
    relax_dgo_gurobi(model, relax_obj_type=0)
    
    relax_lc(tmodel, metabolites_to_ignore=(), solver=None)
        :param metabolites_to_ignore:
        :param in_tmodel:
        :type in_tmodel: pytfa.thermo.ThermoModel:
        :param min_objective_value:


In [46]:
mytfa.delta_g.get_by_id('AIRC3')

-1000.0 <= DG_AIRC3 <= 1000.0

In [35]:
help(relaxed_model.delta_g[0])

Help on DeltaG in module pytfa.optim.variables object:

class DeltaG(ReactionVariable)
 |  DeltaG(reaction, **kwargs)
 |  
 |  Class to represent a DeltaG
 |  
 |  Method resolution order:
 |      DeltaG
 |      ReactionVariable
 |      GenericVariable
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  __slotnames__ = []
 |  
 |  prefix = 'DG_'
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ReactionVariable:
 |  
 |  __init__(self, reaction, **kwargs)
 |      :param id_: will be used to identify the variable
 |          (name will be a concat of this and a prefix)
 |      :param model: the cobra.Model object
 |      :param queue: whether or not to queue the variable for update object
 |      :param scaling_factor: scaling factor used in self.scaled, useful for
 |                              adimensionalisation of constraints
 |      :param kwargs: stuff

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import pytfa
import numpy as np
from optlang.exceptions import SolverError

#from cobra.core.model import SolverNotFound
from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data
from pytfa.optim.relaxation import relax_dgo

import pickle
from pytfa.optim.variables import LogConcentration
#thermo_database = '/projectnb2/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/data/thermo_data.thermodb'
#root_dir = '/projectnb/bioinfor/SEGRE/goldford/CoenzymeSpecificity/pytfa/tests/singleCoenzymeModel.08272021.v2'
thermo_database = '../assets/thermo_data.thermodb'
model_path = '../assets'

CPLEX = 'optlang-cplex'
GUROBI = 'optlang-gurobi'
GLPK = 'optlang-glpk'
solver = GUROBI

# Load reaction DB
print("Loading thermo data...")
thermo_data = load_thermoDB(thermo_database)

print("Done !")
#biomass_rxn = 'BIOMASS_Ec_iJO1366_WT_53p95M'
biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'


model_id = 'iJO1366_WT_semi-unconstrained.json'
#model_id = 'iJO1366.json'

# We import pre-compiled data as it is faster for bigger models
cobra_model = load_json_model(model_path + '/' + model_id)

lexicon = read_lexicon(model_path + '/iJO1366/lexicon.csv')
compartment_data = read_compartment_data(model_path + '/iJO1366/compartment_data.json')

# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

# Annotate the cobra_model
annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)


mytfa.name = 'iJO1366[WT]'
mytfa.solver = solver
mytfa.objective = biomass_rxn

# Solver settings

def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)


## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
print('FBA Solution found : {0:.5g}'.format(fba_value))

# fva = flux_variability_analysis(mytfa)

## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

## Info on the cobra_model
mytfa.print_info()

## Optimality
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value
print('TFA Solution found : {0:.5g}'.format(tfa_value))

# It might happen that the model is infeasible. In this case, we can relax 
# thermodynamics constraints:

if tfa_value < 0.1:
    
    from pytfa.optim.relaxation import relax_dgo
    #mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)
    original_model, mytfa = mytfa, relaxed_model
    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

    print('relaxed TFA Solution found : {0:.5g}'.format(tfa_value))

# Report
print('FBA Solution found : {0:.5g}'.format(fba_value))
print('TFA Solution found : {0:.5g}'.format(tfa_value))

double_coenzyme_models = {}
z = mytfa.copy()
double_coenzyme_models['tmodel'] = z
double_coenzyme_models['tmodel-maxGrowth'] = tfa_value
double_coenzyme_models['tmodel-solution'] = tfa_solution

Loading thermo data...
Done !
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmptto0q2tc.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2021-10-11 10:16:06,048 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-11 10:16:06,270 - thermomodel_None - INFO - # Model preparation starting...


FBA Solution found : 24.714


2021-10-11 10:16:08,453 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 10:16:08,454 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,455 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,456 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 10:16:08,457 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,458 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,636 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 10:16:08,637 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,638 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,638 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 10:16:08,639 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,640 - thermomodel_None - WARNING - Warning : NULL/L
2021-10-11 10:16:08,781 - thermomodel_None - WARNING - Warning : NULL/U
2021-10-11 10:16:08,782 - thermomodel_None - WARNING - Warning :

                       value
key                         
name             iJO1366[WT]
description      iJO1366[WT]
num constraints        15034
num variables          15538
num metabolites         1807
num reactions           2585
                             value
key                               
num metabolites(thermo)       1550
num reactions(thermo)         1824
pct metabolites(thermo)  85.777532
pct reactions(thermo)    70.560928
TFA Solution found : 0
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmppqdtv3qe.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2021-10-11 10:17:02,848 - thermomodel_iJO1366[WT] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp0swdz84e.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2021-10-11 10:17:20,851 - thermomodel_iJO1366[WT] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-11 10:17:28,755 - thermomodel_iJO1366[WT] - INFO - Adding slack constraints
adding slacks: 100%|██████████| 1824/1824 [00:48<00:00, 37.73it/s]
2021-10-11 10:18:18,496 - thermomodel_iJO1366[WT] - INFO - Optimizing slack model
2021-10-11 10:18:26,723 - thermomodel_iJO1366[WT] - INFO - Extracting relaxation
applying slack: 100%|██████████| 2585/2585 [00:00<00:00, 103820.40it/s]
2021-10-11 10:18:26,837 - thermomodel_iJO1366[WT] - INFO - Testing relaxation


Relaxation: 
              lb_in      ub_in  lb_change  ub_change     lb_out     ub_out
AIRC3    -11.941085  -8.988685   0.000000   2.582409 -11.941085  -6.406276
ARGSL     16.770653  18.636729   2.184361   0.000000  14.586293  18.636729
ATPS4rpp  -9.518201  -8.083092   1.452217   0.000000 -10.970418  -8.083092
GALT1     67.754735  69.018800  54.942183   0.000000  12.812552  69.018800
HEPK2     44.872230  45.345690  32.059679   0.000000  12.812552  45.345690
HEPT1     25.810459  27.074524  13.693847   0.000000  12.116612  27.074524
MECDPS    26.135683  30.646876  11.549390   0.000000  14.586293  30.646876
MTHTHFSs  13.297862  14.814659   6.891586   0.000000   6.406276  14.814659
PRAIS     17.442418  22.161249   0.065725   0.000000  17.376694  22.161249
relaxed TFA Solution found : 0.838
FBA Solution found : 24.714
TFA Solution found : 0.838
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpmcgum2zu.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 column

2021-10-11 10:19:04,877 - thermomodel_RelaxedModel iJO1366[WT] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [12]:
single_coenzyme_models

{'tmodel': <ThermoModel RelaxedModel_iJO1366[NAD] at 0x7fa8b246f550>,
 'tmodel-maxGrowth': 0.7247860099621941,
 'tmodel-solution': <Solution 0.725 at 0x7fa8c87d6ee0>}

In [13]:
double_coenzyme_models

{'tmodel': <ThermoModel RelaxedModel_ttmodel at 0x7fa8cb1b7ac0>,
 'tmodel-maxGrowth': 0.8379979489589438,
 'tmodel-solution': <Solution 0.838 at 0x7fa8cb1d1a30>}

In [16]:
filename = 'wt_coenzyme_model.tfa.pkl'
with open(filename,'wb') as handle:
    pickle.dump(double_coenzyme_models,handle)

In [17]:
with open(filename,'rb') as handle:
    dc = pickle.load(handle)

Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpvf8g_yfd.lp
Reading time = 0.07 seconds
: 15034 rows, 15538 columns, 52691 nonzeros
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp6q2voxwm.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros
Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp83l3bkqt.lp
Reading time = 0.05 seconds
: 15034 rows, 15538 columns, 52691 nonzeros


In [ ]:
dc

In [146]:
tmodel = double_coenzyme_models['tmodel'].copy()
tfa_value =  double_coenzyme_models['tmodel-solution'].objective_value

mets = {'id': [] , 'opt-direction': [], 'value': []}
for met in tmodel.get_variables_of_type(LogConcentration):
    tmodel.objective = met + 0
    tmodel.reactions.get_by_id(biomass_rxn).lower_bound = tfa_value - 1e-6
    tmodel.objective.direction = 'max'
    maxm = tmodel.optimize()
    mets['id'].append(met.id)
    mets['opt-direction'].append('max')
    mets['value'].append(maxm.objective_value)
    
    tmodel.objective.direction = 'min'
    maxm = tmodel.optimize()
    mets['id'].append(met.id)
    mets['opt-direction'].append('min')
    mets['value'].append(maxm.objective_value)
    
met_ranges = pd.DataFrame(mets)
met_ranges.to_csv('met_ranges.tfa.wt.maxGrowthRate.08292021.csv')


tmodel = single_coenzyme_models['tmodel'].copy()
tfa_value =  single_coenzyme_models['tmodel-solution'].objective_value


mets = {'id': [] , 'opt-direction': [], 'value': []}
for met in tmodel.get_variables_of_type(LogConcentration):
    tmodel.objective = met + 0
    tmodel.reactions.get_by_id(biomass_rxn).lower_bound = tfa_value - 1e-6
    tmodel.objective.direction = 'max'
    maxm = tmodel.optimize()
    mets['id'].append(met.id)
    mets['opt-direction'].append('max')
    mets['value'].append(maxm.objective_value)
    
    tmodel.objective.direction = 'min'
    maxm = tmodel.optimize()
    mets['id'].append(met.id)
    mets['opt-direction'].append('min')
    mets['value'].append(maxm.objective_value)
    
met_ranges = pd.DataFrame(mets)
met_ranges.to_csv('met_ranges.tfa.sc.maxGrowthRate.08292021.csv')


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpapa5ib7l.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2021-08-29 18:50:07,230 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmp_ak348mq.lp
Reading time = 0.02 seconds
: 1805 rows, 5164 columns, 20264 nonzeros


2021-08-29 22:47:14,520 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [11]:
double_coenzyme_models

{'tmodel': <ThermoModel RelaxedModel_ttmodel at 0x7fa8cb1b7ac0>,
 'tmodel-maxGrowth': 0.8379979489589438,
 'tmodel-solution': <Solution 0.838 at 0x7fa8cb1d1a30>}

In [6]:
from pytfa.analysis.variability import variability_analysis

In [7]:
tmodel = double_coenzyme_models['tmodel'].copy()
tfa_value =  double_coenzyme_models['tmodel-solution'].objective_value
tmodel.reactions.get_by_id(biomass_rxn).lower_bound = tfa_value - 1e-6
varib_results_wt = variability_analysis(tmodel,kind='reactions')

tmodel = single_coenzyme_models['tmodel'].copy()
tfa_value =  single_coenzyme_models['tmodel-solution'].objective_value
tmodel.reactions.get_by_id(biomass_rxn).lower_bound = tfa_value - 1e-6
varib_results_sc = variability_analysis(tmodel,kind='reactions')

Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpsg8mfsfp.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2021-10-08 15:08:03,768 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-08 15:08:10,482 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 2585/2585 [14:26:14<00:00, 20.11s/it]     


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmphfuyp0kr.lp
Reading time = 0.01 seconds
: 1805 rows, 5164 columns, 20264 nonzeros


2021-10-09 10:19:44,107 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-09 10:19:50,552 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 2582/2582 [11:09<00:00,  3.86it/s]


In [10]:
varib_results_sc.to_csv('FVA.sc.results.optim.09Oct2021.csv')
varib_results_wt.to_csv('FVA.wt.results.optim.09Oct2021.csv')

In [ ]:
tmodel = double_coenzyme_models['tmodel'].copy()
#tfa_value =  double_coenzyme_models['tmodel-solution'].objective_value
tmodel.reactions.get_by_id(biomass_rxn).lower_bound = 0.5
varib_results_wt = variability_analysis(tmodel,kind='reactions')

tmodel = single_coenzyme_models['tmodel'].copy()
tfa_value =  single_coenzyme_models['tmodel-solution'].objective_value
tmodel.reactions.get_by_id(biomass_rxn).lower_bound = 0.5
varib_results_sc = variability_analysis(tmodel,kind='reactions')

varib_results_sc.to_csv('FVA.sc.results.grzp5.10Oct2021.csv')
varib_results_wt.to_csv('FVA.wt.results.grzp5.10Oct2021.csv')


Read LP format model from file /var/folders/17/58pxvfhj0gb_wz2nzgrzc6pc0000gn/T/tmpzx0ywf3k.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2021-10-10 12:30:06,954 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-10-10 12:30:17,547 - thermomodel_RelaxedModel iJO1366[NAD] - INFO - Beginning variability analysis for variable of type reactions
maximizing:  10%|▉         | 258/2585 [6:07:41<38:55,  1.00s/it]      

In [15]:
os.getcwd()

'/Users/joshuagoldford/Documents/github/coenzymes/revisions/notebooks'

In [8]:
varib_results_wt.to_csv('FVA.wt.results.31Aug2021.csv')

In [87]:
met_ranges.to_csv('met_ranges.tfa.singleCoenzymeUnconstrained.csv')

In [99]:
#tfa_solution.
np.exp(-2.995732273553991)

0.05000000000000001

In [108]:
# find min/max NADH/NAD ratios
nad = [x for x in tmodel.get_variables_of_type(LogConcentration) if x.id == 'nad_c'][0]
nadh = [x for x in tmodel.get_variables_of_type(LogConcentration) if x.id == 'nadh_c'][0]
tmodel.objective = nadh - nad
#tmodel.reactions.get_by_id(biomass_rxn).lower_bound = tfa_value - 1e-6

tmodel.reactions.get_by_id(biomass_rxn).lower_bound = 0
tmodel.objective.direction = 'max'
max_ratio = tmodel.optimize()

tmodel.objective.direction = 'min'
min_ratio = tmodel.optimize()

0.0008421313800558607

In [110]:
max_ratio

,fluxes,reduced_costs
DM_4CRSOL,0.000179,None
DM_5DRIB,0.000186,None
DM_AACALD,0.000000,None
DM_AMOB,0.000002,None
DM_MTHTHF,0.001076,None
...,...,...
ZN2abcpp,0.000000,None
ZN2t3pp,0.000000,None
ZN2tpp,0.000260,None
ZNabcpp,0.000000,None


In [95]:
tmodel.log_concentration

[-13.815510557964274 <= LC_10fthf_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr120_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr140_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr141_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr160_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr161_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr180_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12dgr181_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12ppd-R_c <= -2.995732273553991,
 -13.815510557964274 <= LC_12ppd-S_c <= -2.995732273553991,
 -13.815510557964274 <= LC_13dpg_c <= -2.995732273553991,
 -13.815510557964274 <= LC_14dhncoa_c <= -2.995732273553991,
 -13.815510557964274 <= LC_14glucan_c <= -2.995732273553991,
 -13.815510557964274 <= LC_15dap_c <= -2.995732273553991,
 -13.815510557964274 <= LC_1ddecg3p_c <= -2.995732273553991,
 -13.815510557964274 <= LC_1hdec9eg3p_c <= -2.995732273553991,
 -13.815510557964274 <= LC_1hdec